# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
text[:100].split()

['Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 "Moe's",
 'Tavern.',
 'Where',
 'the',
 'elite',
 'meet',
 'to',
 'drink.',
 'Bart_Simpson:',
 'Eh,',
 'yeah,',
 'hell']

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    from collections import Counter
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = {v:k for k, v in vocab_to_int.items()}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Had to investigate this error (and it is now answered)
I had copied code from the lesson which had
```
enumerate(vocab, 1)
```
and that basically spoiled [my whole Saturday (see Week 9 column G)](https://docs.google.com/spreadsheets/d/13C8TVCn2jNuUQ3RtjQyF940oQ038Gyd-RwQHNPWbSOg/edit#gid=608870057).

In [5]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables_1(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    from collections import Counter
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    int_to_vocab = {v:k for k, v in vocab_to_int.items()}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables_1)

Tests Passed


In [6]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables_2(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int = {w: i for i, w in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables_2)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [7]:
short_words = {}
vocab_to_int, int_to_vocab = create_lookup_tables(text)
for word in vocab_to_int:
    if len(word)<3:
        if word in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789":
            continue
        short_words[word] = ""
print(short_words)

{'¨': '', '$': '', '³': '', '-': '', ',': '', '£': '', '%': '', '.': '', ':': '', ' ': '', '_': '', ')': '', '"': '', '&': '', '?': '', '¼': '', '\n': '', '\xa0': '', '©': '', 'Ã': '', '€': '', '(': '', ';': '', '¤': '', '!': '', "'": '', '#': '', '/': ''}


In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    # Pass ONLY for symbols = set(['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n'])
#     return {'--': '||long_dash||',
# '$': '||dollar||',
# ',': '||comma||',
# '.': '||period||',
# "'": '||quote||',
# '(': '||left_paren||',
# '¼': '||one_quarter||',
# ')': '||right_paren||',
# '%': '||percent||',
# '€': '||euro||',
# '_': '||underscore||',
# '\xa0': '||non_breaking_space||',
# '¤': '||currency_sign||',
# '©': '||copyright||',
# '/': '||slash||',
# '¨': '||umlaut||',
# '&': '||amp||',
# '#': '||hash||',
# '³': '||third||',
# ';': '||semicolon||',
# '£': '||pound||',
# '?': '||question||',
# '\n': '||new_line||',
# ':': '||colon||',
# '"': '||double_quote||',
# '!': '||exclamation||',
# '-': '||minus||',
# 'Ã': '||a_tilde||',
# ' ': '||space||'}

    return {'--': '||long_dash||',
',': '||comma||',
'.': '||period||',
'(': '||left_paren||',
')': '||right_paren||',
';': '||semicolon||',
'?': '||question||',
'\n': '||new_line||',
'"': '||double_quote||',
'!': '||exclamation||'}

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [9]:
import os
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

In [10]:
text = load_data(data_dir)

token_dict = token_lookup()
for key, token in token_dict.items():
    text = text.replace(key, ' {} '.format(token))

text = text.lower()
text = text.split()

vocab_to_int, int_to_vocab = create_lookup_tables(text)
vocab_to_int_1, int_to_vocab_1 = create_lookup_tables_1(text)
vocab_to_int_2, int_to_vocab_2 = create_lookup_tables_2(text)

# What is so bad about create_lookup_tables_1?

In [11]:
vocab_to_int_2 == vocab_to_int

False

In [12]:
list(vocab_to_int_2.items())[:5]

[("d'", 0),
 ('ducked', 3335),
 ('dealie', 1),
 ('backwards', 3337),
 ('scratcher', 2)]

In [13]:
list(vocab_to_int.items())[:5]

[("d'", 2898),
 ('ducked', 2899),
 ('dealie', 2900),
 ('backwards', 1440),
 ('yammering', 6442)]

In [14]:
int_to_vocab_2 == int_to_vocab

False

In [15]:
list(int_to_vocab_2.items())[:5]

[(0, "d'"), (1, 'dealie'), (2, 'scratcher'), (3, 'covering'), (4, 'loved')]

In [16]:
list(int_to_vocab.items())[:5]

[(0, '||period||'),
 (1, '||new_line||'),
 (2, '||comma||'),
 (3, '||left_paren||'),
 (4, '||right_paren||')]

## Answer: create_lookup_tables_1 does not start at 0

In [17]:
list(int_to_vocab_1.items())[:5]

[(1, '||period||'),
 (2, '||new_line||'),
 (3, '||comma||'),
 (4, '||left_paren||'),
 (5, '||right_paren||')]

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [20]:
int_text[:15]

[6585, 448, 6459, 6663, 2467, 1588, 1530, 1315, 1756, 0, 37, 5097, 2206, 0, 1]

In [21]:
_ = [print(int_to_vocab[item], end=" ") for item in int_text[:15]]

[year date 1989] â© twentieth century fox film corporation ||period|| all rights reserved ||period|| ||new_line|| 

In [22]:
int_text[16:32]

[9, 3, 121, 178, 4, 177, 338, 0, 142, 5, 4867, 536, 13, 151, 0, 1]

In [23]:
_ = [print(int_to_vocab[item], end=" ") for item in int_text[16:32]]

moe_szyslak: ||left_paren|| into phone ||right_paren|| moe's tavern ||period|| where the elite meet to drink ||period|| ||new_line|| 

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [25]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return (input, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [26]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm, lstm])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name= "initial_state")
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [27]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    # tf.contrib.layers.embed_sequence(ids, vocab_size=None, embed_dim=None, unique=False, initializer=None, regularizer=None, trainable=True, scope=None, reuse=None)
    # From https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence
    return tf.contrib.layers.embed_sequence(input_data, 
                                            vocab_size=vocab_size, 
                                            embed_dim=embed_dim)

#     embeddings = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0))
#     return tf.nn.embedding_lookup(embeddings, input_data)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [28]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    return (outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [29]:
EMBED_DIM = 300

In [30]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    inputs = get_embed(input_data, vocab_size, EMBED_DIM)
    outputs, final_state = build_rnn(cell, inputs)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [31]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    int_text = int_text[16:]
    n_batches = (len(int_text)-1)//(batch_size * seq_length)    
    int_text = int_text[:n_batches * batch_size * seq_length + 1]
    int_text_sequences = [int_text[i*seq_length:i*seq_length+seq_length] for i in range(0, n_batches * batch_size)]
    int_text = int_text[1:]
    int_text_targets = [int_text[i*seq_length:i*seq_length+seq_length] for i in range(0, n_batches * batch_size)]
    output = []
    for batch in range(n_batches):
        inputs = []
        targets = []
        for size in range(batch_size):
            inputs.append(int_text_sequences[size * n_batches + batch])
            targets.append(int_text_targets[size * n_batches + batch])
        output.append([inputs, targets])
    return np.array(output)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [32]:
sentences_in_scene = 15
words_in_sentence = 11
words_in_scene = sentences_in_scene * words_in_sentence
words_in_scene

165

In [33]:
total_words = len(int_text[16:])
total_words

69100

In [34]:
scenes = int(total_words / words_in_scene)
scenes

418

In [35]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 119
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = get_batches(int_text, batch_size, seq_length).shape[0]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [36]:
print("Batch Size -> words missed\n")
for i in range(-5,5):
    try_batch_size = batch_size + i
    batches = get_batches(int_text, try_batch_size, seq_length)
    flag = ""
    if i == 0:
        flag = "<<< Current choice. Should be minimum words missed."
    print("{:>10}   {:>5} {}".format(try_batch_size, total_words - try_batch_size * seq_length * batches.shape[0], flag))

Batch Size -> words missed

       114     700 
       115     100 
       116    1820 
       117    1240 
       118     660 
       119      80 <<< Current choice. Should be minimum words missed.
       120    1900 
       121    1340 
       122     780 
       123     220 


In [37]:
batches = get_batches(int_text, batch_size, seq_length)
print('(number of batches, 2, batch size, sequence length)')
print(batches.shape)

total_words = len(int_text[16:])
print('total_words   {:>6}'.format(total_words))

words_batched = batch_size * seq_length * batches.shape[0]
print('words_batched {:>6}'.format(words_batched))

words_missed = total_words - words_batched
print('words_missed  {:>6}'.format(words_missed))

# _ = [print(int_to_vocab[item], end=" ") for item in int_text[words_batched+16:]]

(number of batches, 2, batch size, sequence length)
(29, 2, 119, 20)
total_words    69100
words_batched  69020
words_missed      80


## First in first batch

In [38]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[0][0][0][:]]

moe_szyslak: ||left_paren|| into phone ||right_paren|| moe's tavern ||period|| where the elite meet to drink ||period|| ||new_line|| bart_simpson: eh ||comma|| yeah 

In [39]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[0][1][0][:]]

||left_paren|| into phone ||right_paren|| moe's tavern ||period|| where the elite meet to drink ||period|| ||new_line|| bart_simpson: eh ||comma|| yeah ||comma|| 

## First in second batch

In [40]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[1][0][0][:]]

||comma|| hello ||comma|| is mike there ||question|| last name ||comma|| rotch ||period|| ||new_line|| moe_szyslak: ||left_paren|| into phone ||right_paren|| hold on 

In [41]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[1][1][0][:]]

hello ||comma|| is mike there ||question|| last name ||comma|| rotch ||period|| ||new_line|| moe_szyslak: ||left_paren|| into phone ||right_paren|| hold on ||comma|| 

## Last in last batch

In [42]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[-1][0][-1][:]]

i can find a man like that ||period|| ||left_paren|| thinks a beat ||right_paren|| patty ||comma|| are you throwing away your 

In [43]:
_ = [print(int_to_vocab[item], end=" ") for item in batches[-1][1][-1][:]]

can find a man like that ||period|| ||left_paren|| thinks a beat ||right_paren|| patty ||comma|| are you throwing away your last 

### Build the Graph
Build the graph using the neural network you implemented.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [45]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/29   train_loss = 8.822
Epoch   1 Batch    0/29   train_loss = 6.623
Epoch   2 Batch    0/29   train_loss = 6.205
Epoch   3 Batch    0/29   train_loss = 6.101
Epoch   4 Batch    0/29   train_loss = 5.813
Epoch   5 Batch    0/29   train_loss = 5.180
Epoch   6 Batch    0/29   train_loss = 4.848
Epoch   7 Batch    0/29   train_loss = 4.672
Epoch   8 Batch    0/29   train_loss = 4.496
Epoch   9 Batch    0/29   train_loss = 4.355
Epoch  10 Batch    0/29   train_loss = 4.246
Epoch  11 Batch    0/29   train_loss = 4.131
Epoch  12 Batch    0/29   train_loss = 4.016
Epoch  13 Batch    0/29   train_loss = 3.914
Epoch  14 Batch    0/29   train_loss = 3.811
Epoch  15 Batch    0/29   train_loss = 3.712
Epoch  16 Batch    0/29   train_loss = 3.620
Epoch  17 Batch    0/29   train_loss = 3.511
Epoch  18 Batch    0/29   train_loss = 3.432
Epoch  19 Batch    0/29   train_loss = 3.345
Epoch  20 Batch    0/29   train_loss = 3.245
Epoch  21 Batch    0/29   train_loss = 3.171
Epoch  22 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [48]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0") 
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0") 
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return  (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [49]:
import random

def weighted_choice(choices):
    # From http://stackoverflow.com/questions/3679694/a-weighted-version-of-random-choice
    total = sum(w for c, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"
    
def pick_from_top_5(choices):
    top5 = []
    for i in range(min(len(choices), 5)):
        index = np.argmax(choices)
        top5.append((index, choices[index]))
        choices.itemset(index, 0) # Avoid picking this index as argmax again
    return weighted_choice(top5)

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return int_to_vocab[pick_from_top_5(probabilities)]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [50]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: oh, the heat's been on since them bush girls were in here.
homer_simpson: since i think about time they saw this.
homer_simpson: oh, moe. maybe i've learned your best drink it is... i couldn't you find the hand for a couple.
moe_szyslak:(to moe) hey, homer, if you got it," i aren't a glass up to my guy who shows up in the bar.
moe_szyslak:(laughs) oh marge, i can't take that out?
lenny_leonard: that's the best break, the low...
moe_szyslak: barney, look! that was the longest gets every beer / and city third the house!
homer_simpson: hey, homer. here i come up, homer. i am changing the other day and water?
moe_szyslak: hey, you don't have to wait my new guy i ever. who am i got a big idea?
moe_szyslak:(starts to watch me go to the bar and die. you know, when they said no girl somewhere!
ken:


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.